Nhận dạng đường chỉ tay sử dụng ***đặc trưng LBPH*** và không gian ***PCA*** cùng với ***LDA***.

*   *Đặc trưng LBPH* được rút trích bằng cách tính *giá trị LBP* cho từng điểm ảnh. Sau đó, *ma trận LBP* kết quả được chia thành các vùng bằng nhau. Tính lược đồ cho mỗi vùng và nối lại với nhau thành một vector đặc trưng.

In [1]:
%cd /content/drive/My Drive/Notebooks/Projects/Palmprint Recognition/

/content/drive/My Drive/Notebooks/Projects/Palmprint Recognition


In [2]:
import cv2
import csv
import numpy as np

from skimage import feature
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

***Đọc dữ liệu:*** Dữ liệu gồm $N = 1301$ ảnh ROI có kích thước *150 x 150* đã được tiền xử lý và lưu dưới dạng các vector có $D = 22500$ chiều, nên ma trận dữ liệu $X$ có kích thước $(1301, 22500)$, và vector nhãn tương ứng $Y$ có kích thước $(1301, 1)$.

In [3]:
fX = open("IITD Palmprint V1/X.csv", "r")
reader = csv.reader(fX)
X = np.array(list(reader)).astype(np.uint8)
fX.close()

In [4]:
fY = open("IITD Palmprint V1/Y.csv", "r")
reader = csv.reader(fY)
Y = np.array(list(reader)).astype(np.int64)
fY.close()

***Tham số:*** *LBP* được sử dụng trong đồ án này là $LBP_{p,r}^{riu2}$ với $p = 16$ và $r = 2$.

In [5]:
p = 16
r = 2

maxVal = p + 2

***Rút trích đặc trưng:***

1.   Tính giá trị $LBP_{16,2}^{riu2}$ cho mỗi điểm ảnh.
2.   Chia ảnh kết quả thành các vùng *25 x 25*.
3.   Tính lược đồ cho mỗi vùng và nối lại thành một vector đặc trưng.

In [6]:
def LBPH(img):
  # Calculate LBP value for each pixel
  lbp = feature.local_binary_pattern(img, p, r, method = "uniform").astype(np.uint8)

  # Feature vector is a ((p + 2) values * 36 sub-regions) dimensions vector
  vector = np.zeros(((p + 2) * 36,))

  # Split the LBP matrix to 36 sub-regions
  for i in range(6):
    for j in range(6):
      startR = 25 * i
      startC = 25 * j

      # Calculate histogram for each sub-region
      sub = lbp[startR:startR + 25, startC:startC + 25]
      hist = cv2.calcHist([sub], [0], None, [maxVal], [0, maxVal]).reshape(-1)
      
      # Place the histogram in appropriate position
      pos = (i * 6 + j) * maxVal
      vector[pos:pos + maxVal] = hist

  # Normalize final feature vector
  length = np.sqrt(np.dot(vector, vector))
  vector /= length

  return vector

In [7]:
X_lbp = np.zeros((X.shape[0], (p + 2) * 36))

for i in range(X.shape[0]):
  img = X[i].reshape(150, 150)
  X_lbp[i] = LBPH(img)

***Tạo tập học và tập kiểm thử:*** tỉ lệ mẫu của hai tập này là *0.85/0.15*.

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X_lbp, Y, test_size = 0.15)

***Tạo không gian đặc trưng:*** giảm số chiều bằng *PCA* sau đó áp dụng *LDA* để các lớp tách biệt nhau nhất có thể.

In [9]:
pca = PCA(n_components = 100)
Z_train = pca.fit_transform(X_train)

lda = LinearDiscriminantAnalysis()
Z_train = lda.fit_transform(Z_train, Y_train.reshape(-1))

***Xây dựng bộ phân lớp và kiểm thử:*** dùng bộ phân lớp *SVM* với *kernel tuyến tính* để dự đoán nhãn cho tập test.

In [10]:
svc = SVC(kernel = "linear")
svc.fit(Z_train, Y_train.reshape(-1))

Z_test = pca.transform(X_test)
Z_test = lda.transform(Z_test)

Y_predict = svc.predict(Z_test).reshape(-1, 1)

print("Accuracy: {} %".format(np.round(np.mean(Y_predict == Y_test) * 100, 2)))

Accuracy: 90.82 %
